# 108: Feature Stores with Feast

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Understand** the feature store architecture and why it solves training-serving skew
- **Implement** Feast feature store with both batch and real-time features
- **Build** feature versioning and lineage tracking for reproducibility
- **Apply** feature stores to semiconductor test data pipelines
- **Evaluate** online vs offline feature retrieval performance and consistency

## 📚 What is a Feature Store?

A feature store is a centralized repository that manages, versions, and serves ML features consistently across training and production. Without a feature store, teams duplicate feature engineering logic (SQL in training notebooks, Python in production APIs), causing subtle bugs where training features ≠ serving features. Feature stores solve this by computing features once, storing them with metadata (version, timestamp, lineage), and serving them through unified APIs.

Feast (Feature Store) is an open-source solution providing **online store** (Redis/DynamoDB for <10ms serving) and **offline store** (BigQuery/Snowflake for batch training). Features are defined declaratively—you specify transformations once, and Feast handles materialization to both stores. Point-in-time correctness ensures no data leakage: when retrieving historical features for training at timestamp T, you only get data available before T.

In semiconductor manufacturing, feature stores standardize parametric test features across yield prediction, binning, and test time optimization models. Instead of each team extracting "average Vdd over last 10 wafers" differently, the feature is computed once from STDF files, versioned (v1.2.3), and reused. When a new tester is added, update the feature definition centrally and all models get consistent data.

**Why Feature Stores?**
- ✅ **Consistency**: Training features = serving features (eliminate training-serving skew)
- ✅ **Reusability**: Define "wafer_avg_vdd" once, use in 10 models
- ✅ **Versioning**: Reproduce model trained 6 months ago with exact feature definitions
- ✅ **Speed**: <10ms online serving, pre-computed aggregations
- ✅ **Governance**: Track feature lineage, who uses which features, data quality

## 🏭 Post-Silicon Validation Use Cases

**Use Case 1: Parametric Feature Standardization**
- **Problem**: 5 teams compute "device power" differently (Vdd×Idd vs max instantaneous vs average)
- **Solution**: Define "device_power_avg" feature in Feast, all teams use same calculation
- **Feature**: `vdd_mean * idd_mean` computed from STDF, updated daily
- **Impact**: Eliminated 3 bugs where models disagreed on device classification
- **Value**: 100% feature consistency across 12 ML models

**Use Case 2: Real-Time Wafer Context**
- **Problem**: Yield model needs last 50 wafers' statistics, slow to compute on-the-fly
- **Solution**: Feast online store with streaming aggregations (Kafka → Redis)
- **Feature**: Rolling 50-wafer mean/std for Vdd, Idd, frequency (updated every wafer)
- **Impact**: Reduced serving latency from 800ms to 12ms
- **Value**: Enabled real-time yield prediction during test (catch bad wafers early)

**Use Case 3: Feature Versioning for Reproducibility**
- **Problem**: Model trained 3 months ago, need to reproduce results for audit
- **Solution**: Feast feature versioning with point-in-time correctness
- **Feature**: `wafer_spatial_features_v2` (center vs edge die statistics)
- **Impact**: Reproduced training data exactly, passed compliance audit
- **Value**: Regulatory compliance for automotive qualification

**Use Case 4: Multi-Fab Feature Harmonization**
- **Problem**: 3 fabs measure temperature differently (°C, °F, different sensors)
- **Solution**: Feast feature transformations normalize to standard units
- **Feature**: `test_temp_normalized` (all fabs → Celsius, calibrated)
- **Impact**: Single model works across all fabs (was 3 separate models)
- **Value**: 60% reduction in model maintenance overhead

## 🔄 Feature Store Architecture

```mermaid
graph TB
    A[Raw Data Sources] --> B[Feature Definitions]
    B --> C[Feast SDK]
    
    A1[STDF Files] --> A
    A2[SQL Database] --> A
    A3[Kafka Streams] --> A
    
    C --> D[Offline Store]
    C --> E[Online Store]
    
    D --> D1[Parquet/BigQuery]
    D1 --> F[Training Pipeline]
    
    E --> E1[Redis/DynamoDB]
    E1 --> G[Real-Time Serving]
    
    F --> H[Model Training]
    H --> I[Model Registry]
    
    G --> J[Prediction API]
    J --> K[Production Traffic]
    
    B --> L[Feature Registry]
    L --> M[Metadata]
    M --> M1[Version]
    M --> M2[Owner]
    M --> M3[Lineage]
    
    style A fill:#e1f5ff
    style D1 fill:#e1ffe1
    style E1 fill:#fff5e1
    style K fill:#ffe1f5
```

## 📊 Learning Path Context

**Prerequisites:**
- **041**: Model Evaluation - Understanding feature importance
- **091**: SQL Advanced - Feature extraction from databases
- **103**: Advanced Feature Engineering - Complex feature transformations

**This Notebook (108):**
- Feast installation and configuration
- Feature definition and registration
- Offline retrieval for training
- Online serving for production
- Point-in-time correctness

**Next Steps:**
- **109**: ML Pipelines with Airflow - Automated feature materialization
- **131**: Cloud Deployment - Scalable feature store infrastructure

---

Let's build centralized feature infrastructure! 🗄️

## 1. Setup and Imports

**Note:** This notebook demonstrates feature store concepts. Full Feast requires infrastructure (Redis, file storage). We'll simulate the core workflows.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

# Random seed
np.random.seed(42)

print("✅ Feature store environment ready!")
print("\n📝 Note: Full Feast setup requires:")
print("   - pip install feast")
print("   - Redis for online store")
print("   - Parquet/BigQuery for offline store")
print("\n   This notebook simulates core concepts.")

## 2. Simulate Raw STDF Data Stream

**Purpose:** Generate time-series semiconductor test data as source for features.

**Key Points:**
- **Entity**: `device_id` (primary key for feature lookups)
- **Timestamp**: Event time for point-in-time correctness
- **Raw measurements**: Vdd, Idd, freq, temp (will transform into features)
- **Why this matters**: Features must handle streaming data with timestamps

In [ ]:
# Generate 30 days of test data (streaming simulation)
start_date = datetime(2025, 11, 1)
n_devices = 5000
dates = [start_date + timedelta(days=i//200) for i in range(n_devices)]

# Raw STDF measurements
raw_data = pd.DataFrame({
    'device_id': [f'DEV_{i:06d}' for i in range(n_devices)],
    'event_timestamp': dates,
    'wafer_id': [f'W{i//100:04d}' for i in range(n_devices)],
    'die_x': np.random.randint(0, 30, n_devices),
    'die_y': np.random.randint(0, 30, n_devices),
    'vdd_raw': np.random.normal(1.2, 0.08, n_devices),
    'idd_raw': np.random.normal(50, 8, n_devices),
    'freq_raw': np.random.normal(2000, 150, n_devices),
    'temp_raw': np.random.normal(85, 12, n_devices),
    'vth_raw': np.random.normal(0.4, 0.03, n_devices),
    'test_time_ms': np.random.uniform(50, 500, n_devices)
})

print(f"Raw STDF data: {len(raw_data)} devices over 30 days")
print(f"Date range: {raw_data['event_timestamp'].min()} to {raw_data['event_timestamp'].max()}")
print(f"\nSample raw data:")
print(raw_data.head())

## 3. Define Feature Transformations

**Purpose:** Transform raw measurements into ML-ready features (the core of feature stores).

**Key Points:**
- **Feature 1**: Device-level aggregations (power, normalized metrics)
- **Feature 2**: Wafer-level rolling statistics (last 50 devices)
- **Feature 3**: Spatial features (distance from wafer center)
- **Why this matters**: Centralized definitions prevent inconsistencies

In [ ]:
# Feature Set 1: Device-level features (fast, no aggregation)
def compute_device_features(df):
    """Transform raw device measurements into ML features."""
    features = df.copy()
    
    # Power consumption
    features['power_watts'] = features['vdd_raw'] * features['idd_raw'] / 1000
    
    # Normalized metrics (Z-score normalization)
    features['vdd_normalized'] = (features['vdd_raw'] - 1.2) / 0.08
    features['idd_normalized'] = (features['idd_raw'] - 50) / 8
    
    # Performance ratio
    features['freq_per_watt'] = features['freq_raw'] / features['power_watts']
    
    # Spatial position
    features['distance_from_center'] = np.sqrt(
        (features['die_x'] - 15)**2 + (features['die_y'] - 15)**2
    )
    
    return features

# Feature Set 2: Wafer-level rolling aggregations (requires historical context)
def compute_wafer_rolling_features(df, window=50):
    """Compute rolling statistics per wafer (simulates online store aggregations)."""
    df_sorted = df.sort_values('event_timestamp')
    
    # Rolling mean/std for last N devices on same wafer
    wafer_features = []
    for wafer_id in df_sorted['wafer_id'].unique():
        wafer_data = df_sorted[df_sorted['wafer_id'] == wafer_id].copy()
        
        wafer_data['wafer_vdd_rolling_mean'] = (
            wafer_data['vdd_raw'].rolling(window=window, min_periods=1).mean()
        )
        wafer_data['wafer_vdd_rolling_std'] = (
            wafer_data['vdd_raw'].rolling(window=window, min_periods=1).std()
        )
        wafer_data['wafer_power_rolling_mean'] = (
            (wafer_data['vdd_raw'] * wafer_data['idd_raw'] / 1000)
            .rolling(window=window, min_periods=1).mean()
        )
        
        wafer_features.append(wafer_data)
    
    return pd.concat(wafer_features, ignore_index=True)

# Apply transformations
print("Computing device-level features...")
device_features = compute_device_features(raw_data)

print("Computing wafer-level rolling features...")
wafer_rolling_features = compute_wafer_rolling_features(device_features)

print(f"\n✅ Features computed for {len(wafer_rolling_features)} devices")
print(f"\nFeature columns: {wafer_rolling_features.columns.tolist()}")
print(f"\nSample features:")
print(wafer_rolling_features[[
    'device_id', 'power_watts', 'freq_per_watt', 
    'wafer_vdd_rolling_mean', 'distance_from_center'
]].head())

## 3. Feast Feature Registry Setup

**Purpose:** Define feature schemas and metadata for centralized feature management.

**Key Points:**
- **Entity**: Primary key for feature lookup (e.g., `device_id`, `user_id`)
- **Feature View**: Logical grouping of features from same data source
- **TTL (Time-To-Live)**: How long features stay in online store before refresh
- **Batch Source**: Historical data (Parquet/BigQuery) for training

**Why This Matters:** Feature registry prevents duplicate feature engineering across teams and ensures version consistency.

In [ ]:
# Simulate Feast feature definitions (in practice, these are defined in feature_store.yaml)
from dataclasses import dataclass
from typing import List

@dataclass
class FeatureView:
    name: str
    entities: List[str]
    features: List[str]
    ttl_hours: int
    description: str

# Define feature views
device_features_view = FeatureView(
    name="device_parametric_features",
    entities=["device_id"],
    features=["Vdd", "Idd", "Frequency", "Power", "Temperature"],
    ttl_hours=168,  # 1 week
    description="Real-time parametric test measurements"
)

wafer_agg_features_view = FeatureView(
    name="wafer_aggregate_features",
    entities=["wafer_id"],
    features=["wafer_avg_yield", "wafer_test_time_p95", "defect_density"],
    ttl_hours=720,  # 30 days
    description="Wafer-level aggregated statistics"
)

print("Feature Registry Initialized:")
print(f"  - {device_features_view.name}: {len(device_features_view.features)} features")
print(f"  - {wafer_agg_features_view.name}: {len(wafer_agg_features_view.features)} features")
print(f"\nFeature freshness guaranteed for {device_features_view.ttl_hours} hours (device-level)")

## 4. Offline Store: Historical Feature Retrieval

**Purpose:** Retrieve point-in-time correct features for training data generation.

**Key Points:**
- **Point-in-Time Correctness**: Features as they existed at training time (no data leakage)
- **Entity Join**: Match features to training labels by entity ID + timestamp
- **Batch Retrieval**: Efficient loading of 100K+ samples for model training
- **Temporal Consistency**: Training data reflects real-world deployment constraints

**Why This Matters:** Prevents train-serve skew by ensuring training features match production feature computation exactly.

In [ ]:
# Simulate offline feature retrieval (point-in-time correct)
# In real Feast: store.get_historical_features(entity_df, features)

# Create training entity dataframe (device IDs + timestamps)
training_entities = pd.DataFrame({
    'device_id': raw_data['device_id'].values[:600],
    'event_timestamp': pd.to_datetime('2024-01-15') + pd.to_timedelta(np.arange(600), unit='h')
})

# Simulate feature join (in practice, Feast does point-in-time join automatically)
offline_features = training_entities.merge(
    device_features[['device_id', 'Vdd', 'Idd', 'Frequency', 'Power', 'Temperature']],
    on='device_id',
    how='left'
)

# Add wafer-level features
wafer_mapping = raw_data[['device_id', 'wafer_id']].drop_duplicates()
offline_features = offline_features.merge(wafer_mapping, on='device_id', how='left')
offline_features = offline_features.merge(
    wafer_features[['wafer_id', 'wafer_avg_yield', 'wafer_test_time_p95', 'defect_density']],
    on='wafer_id',
    how='left'
)

print(f"Offline Features Retrieved:")
print(f"  Samples: {len(offline_features)}")
print(f"  Device-level features: 5 (Vdd, Idd, Frequency, Power, Temperature)")
print(f"  Wafer-level features: 3 (avg_yield, test_time_p95, defect_density)")
print(f"\nSample (first 3 rows):")
print(offline_features.head(3))

## 5. Online Store: Real-Time Feature Serving

**Purpose:** Low-latency feature retrieval for production inference (< 10ms).

**Key Points:**
- **Redis/DynamoDB Backend**: In-memory key-value stores for sub-millisecond lookups
- **Materialization**: Pre-compute features from batch sources and cache in online store
- **Freshness**: TTL-based expiration ensures stale features are refreshed
- **Read Pattern**: `get_online_features(entity_ids, feature_names)` → Dict

**Why This Matters:** Production models need features in milliseconds, not minutes. Online stores bridge batch computation and real-time serving.

In [ ]:
# Simulate online feature store (Redis-like key-value store)
online_store = {}

# Materialize features to online store (in practice: feast materialize-incremental)
for _, row in device_features.iterrows():
    key = f"device:{row['device_id']}"
    online_store[key] = {
        'Vdd': row['Vdd'],
        'Idd': row['Idd'],
        'Frequency': row['Frequency'],
        'Power': row['Power'],
        'Temperature': row['Temperature']
    }

# Simulate real-time feature retrieval
def get_online_features(device_id):
    """Simulate low-latency feature lookup (< 10ms in production)."""
    key = f"device:{device_id}"
    return online_store.get(key, {})

# Example: Model inference at runtime
test_device_id = 'D001'
features = get_online_features(test_device_id)

print(f"Online Feature Retrieval (Simulated):")
print(f"  Device ID: {test_device_id}")
print(f"  Features Retrieved: {features}")
print(f"\n✅ Latency: ~2ms (Redis lookup in production)")
print(f"Total devices in online store: {len(online_store)}")

## 6. Feature Versioning & Lineage

**Purpose:** Track feature transformations and schema changes over time for reproducibility.

**Key Points:**
- **Version Control**: Git-like tracking for feature definitions (v1, v2, v3)
- **Lineage**: Trace feature back to raw data sources and transformations
- **Backward Compatibility**: Old models can still query old feature versions
- **Audit Trail**: Compliance requirement for regulated industries (finance, healthcare)

**Why This Matters:** When model performance changes, knowing which feature version was used is critical for debugging.

In [ ]:
# Simulate feature versioning
feature_versions = {
    'v1.0': {
        'name': 'device_parametric_features_v1',
        'features': ['Vdd', 'Idd', 'Frequency'],  # Original 3 features
        'created_at': '2023-06-01',
        'description': 'Initial parametric features'
    },
    'v1.1': {
        'name': 'device_parametric_features_v1.1',
        'features': ['Vdd', 'Idd', 'Frequency', 'Power'],  # Added Power
        'created_at': '2023-09-15',
        'description': 'Added computed Power feature'
    },
    'v2.0': {
        'name': 'device_parametric_features_v2',
        'features': ['Vdd', 'Idd', 'Frequency', 'Power', 'Temperature'],  # Added Temperature
        'created_at': '2024-01-10',
        'description': 'Added Temperature sensor data (breaking change: requires new data pipeline)'
    }
}

# Lineage tracking (simplified)
feature_lineage = {
    'Power': {
        'source': 'Computed from Vdd * Idd / 1000',
        'dependencies': ['Vdd', 'Idd'],
        'transformation': 'real-time computation'
    },
    'wafer_avg_yield': {
        'source': 'Aggregation of device Pass/Fail',
        'dependencies': ['Pass (per device)'],
        'transformation': 'batch aggregation (daily)'
    }
}

print("Feature Version History:")
for version, metadata in feature_versions.items():
    print(f"\n  {version} ({metadata['created_at']}):")
    print(f"    Features: {metadata['features']}")
    print(f"    Description: {metadata['description']}")

print("\n\nFeature Lineage Example:")
print(f"  Feature: Power")
print(f"    Source: {feature_lineage['Power']['source']}")
print(f"    Dependencies: {feature_lineage['Power']['dependencies']}")
print(f"\n💡 When model v1.5 trained on 2023-10-01, it used feature version v1.1")

## 7. Visualization: Feature Store Architecture

**Purpose:** Understand data flow from raw sources to model serving.

**Key Points:**
- **Dual-Path**: Batch (offline) for training, streaming (online) for inference
- **Materialization**: Background job syncs offline → online store
- **Consistency**: Same feature definitions used in training and serving

**Why This Matters:** Visualizing the architecture prevents common mistakes like using different feature logic in training vs production.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Feature Store Metrics & Architecture', fontsize=16, fontweight='bold')

# 1. Feature freshness (simulated staleness distribution)
freshness_hours = np.random.exponential(12, 1000)  # Exponential distribution
axes[0, 0].hist(freshness_hours, bins=30, color='skyblue', edgecolor='black')
axes[0, 0].axvline(x=24, color='red', linestyle='--', label='SLA: 24 hours')
axes[0, 0].set_title('Feature Freshness Distribution')
axes[0, 0].set_xlabel('Staleness (hours)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].legend()

# 2. Feature coverage (% of entities with complete features)
feature_coverage = {
    'Vdd': 98.5,
    'Idd': 97.2,
    'Frequency': 99.1,
    'Power': 96.8,
    'Temperature': 94.3
}
axes[0, 1].barh(list(feature_coverage.keys()), list(feature_coverage.values()), color='lightgreen', edgecolor='black')
axes[0, 1].axvline(x=95, color='red', linestyle='--', label='Target: 95%')
axes[0, 1].set_title('Feature Coverage (%)')
axes[0, 1].set_xlabel('Coverage (%)')
axes[0, 1].set_xlim(90, 100)
axes[0, 1].legend()

# 3. Online store latency (simulated p50, p95, p99)
latencies = {
    'p50': 2.3,
    'p95': 8.1,
    'p99': 15.4
}
axes[1, 0].bar(latencies.keys(), latencies.values(), color=['green', 'orange', 'red'], edgecolor='black')
axes[1, 0].axhline(y=10, color='red', linestyle='--', label='SLA: 10ms')
axes[1, 0].set_title('Online Store Latency')
axes[1, 0].set_ylabel('Latency (ms)')
axes[1, 0].legend()

# 4. Feature usage (how many models use each feature)
feature_usage = {
    'Vdd': 5,
    'Idd': 4,
    'Frequency': 6,
    'Power': 3,
    'Temperature': 2
}
axes[1, 1].bar(feature_usage.keys(), feature_usage.values(), color='coral', edgecolor='black')
axes[1, 1].set_title('Feature Reuse Across Models')
axes[1, 1].set_ylabel('# of Models Using Feature')
axes[1, 1].set_xlabel('Feature Name')

plt.tight_layout()
plt.show()

print("\n📊 Feature Store Health Summary:")
print(f"  Average Freshness: {np.mean(freshness_hours):.1f} hours")
print(f"  Features Below Coverage Target: {sum([1 for v in feature_coverage.values() if v < 95])}")
print(f"  p99 Latency: {latencies['p99']:.1f}ms (SLA: < 10ms ⚠️)")
print(f"  Most Reused Feature: Frequency (used by {feature_usage['Frequency']} models)")

## 🚀 Real-World Project Templates

Build production feature stores using these architectures:

### 1️⃣ **Post-Silicon Test Feature Platform**
- **Objective**: Centralize parametric test features for yield prediction across 20+ models  
- **Data**: STDF files (10M devices/month), wafer maps, test program versions  
- **Success Metric**: Reduce feature engineering time from 2 weeks → 2 days per model  
- **Features**: Device-level (Vdd, Idd, freq), wafer-level (spatial stats), lot-level (trends)  
- **Tech Stack**: Feast, S3 (offline), Redis (online), Spark for aggregations, Airflow orchestration

### 2️⃣ **E-Commerce Personalization Feature Store**
- **Objective**: Real-time user/product features for recommendation engine  
- **Data**: Clickstream (5M events/hour), purchase history, inventory (100K SKUs)  
- **Success Metric**: < 5ms p99 latency for feature retrieval, 99.9% uptime  
- **Features**: User (lifetime value, category preferences), Product (trending score, inventory)  
- **Tech Stack**: Tecton, DynamoDB, Kinesis, Lambda for real-time transformations

### 3️⃣ **Fraud Detection Feature Mesh**
- **Objective**: Share fraud indicators across credit card, account takeover, and loan fraud models  
- **Data**: Transactions (1M/day), device fingerprints, IP geolocation, social graph  
- **Success Metric**: Block $5M fraudulent transactions/month with <0.5% false positives  
- **Features**: Transaction velocity (1hr/24hr), account age, geolocation risk score  
- **Tech Stack**: Hopsworks, Kafka, Flink for streaming aggregations, PostgreSQL

### 4️⃣ **Autonomous Driving Perception Features**
- **Objective**: Sensor fusion features for object detection models in real-time  
- **Data**: Camera frames (30 FPS), LiDAR point clouds, GPS/IMU (100Hz)  
- **Success Metric**: < 50ms end-to-end latency from sensor → prediction  
- **Features**: Object bounding boxes, lane curvature, relative velocity, weather conditions  
- **Tech Stack**: Custom C++ feature store, ROS topics, Redis, CUDA for GPU transforms

### 5️⃣ **Healthcare Patient Risk Scoring**
- **Objective**: HIPAA-compliant feature store for readmission/mortality prediction  
- **Data**: EHR records (vitals, labs, medications), 500K patients, 10 years history  
- **Success Metric**: Predict 30-day readmission with AUROC > 0.82  
- **Features**: Lab trends (creatinine slope), medication adherence, comorbidity index  
- **Tech Stack**: Feast on-prem, HIPAA-compliant S3, Snowflake, encrypted Redis

### 6️⃣ **Financial Trading Signal Features**
- **Objective**: Sub-millisecond feature serving for algorithmic trading  
- **Data**: Market tick data (1M ticks/sec), order book depth, news sentiment  
- **Success Metric**: < 1ms p50 latency, 99.999% availability  
- **Features**: Price momentum (5s/1min/5min), volatility, bid-ask spread, correlation matrix  
- **Tech Stack**: Custom in-memory C++ store, kdb+/q, FPGA acceleration, co-location

### 7️⃣ **Supply Chain Demand Forecasting**
- **Objective**: Centralized feature platform for 100+ SKU forecast models  
- **Data**: POS sales (10K stores), inventory, promotions, weather, holidays  
- **Success Metric**: Reduce stockouts by 30%, overstock by 25%  
- **Features**: Sales trends (7d/30d/1y), promotional lift, weather impact scores  
- **Tech Stack**: Databricks Feature Store, Delta Lake, Spark, Redshift

### 8️⃣ **Smart Grid Load Forecasting**
- **Objective**: Regional energy demand features for grid balancing  
- **Data**: Smart meter readings (1M households, 15min intervals), weather, events  
- **Success Metric**: < 3% MAPE for 24-hour ahead forecast  
- **Features**: Historical load patterns, temperature lags, day-of-week/holiday indicators  
- **Tech Stack**: Feast, TimescaleDB, Kafka, Prophet for trend features

## 🎯 Key Takeaways

### What is a Feature Store?
A centralized platform for storing, managing, and serving ML features across offline (training) and online (inference) environments with consistency guarantees.

### Why Feature Stores?
- **Eliminate Train-Serve Skew**: Same feature logic in training and production
- **Feature Reuse**: Teams share features instead of duplicating engineering effort
- **Versioning**: Track feature changes and ensure reproducibility
- **Performance**: Low-latency serving (< 10ms) for real-time models

### Core Components

| **Component** | **Purpose** | **Technology Examples** |
|--------------|------------|------------------------|
| **Offline Store** | Historical features for training | S3, BigQuery, Snowflake, Delta Lake |
| **Online Store** | Real-time feature serving | Redis, DynamoDB, Cassandra |
| **Feature Registry** | Metadata (schemas, owners, versions) | PostgreSQL, MySQL, SQLite |
| **Transformation Engine** | Feature computation logic | Spark, Flink, Pandas, SQL |
| **Materialization** | Sync offline → online store | Airflow, Prefect, Kubernetes CronJobs |

### Feast Architecture Patterns

**Batch Features (Offline Store):**
```python
# Define feature view
@feast.feature_view(
    entities=["device_id"],
    ttl=timedelta(weeks=1),
    source=ParquetSource(path="s3://features/device_parametric.parquet")
)
def device_features(df: DataFrame):
    return df[["Vdd", "Idd", "Frequency", "Power", "Temperature"]]

# Training: Point-in-time correct retrieval
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=["device_features:Vdd", "device_features:Frequency"]
).to_df()
```

**Real-Time Features (Online Store):**
```python
# Materialize to Redis
store.materialize_incremental(end_date=datetime.now())

# Inference: Low-latency lookup
features = store.get_online_features(
    entity_rows=[{"device_id": "D001"}],
    features=["device_features:Vdd", "device_features:Temperature"]
).to_dict()
```

### Point-in-Time Correctness
Ensures no data leakage by joining features as they existed at historical timestamps:
- Training on 2023-09-01 uses features computed ≤ 2023-09-01
- Prevents "future peeking" (using 2023-10-01 data to predict 2023-09-01 labels)

### Common Pitfalls
- ❌ **Inconsistent Transformations**: Training uses pandas, serving uses SQL → different results
- ❌ **Stale Online Features**: Forgetting to materialize → serving outdated features
- ❌ **No Feature Monitoring**: Drift goes unnoticed until model degrades
- ❌ **Over-Engineering**: Small projects don't need full feature store (start simple)

### When to Use Feature Stores
- ✅ **Multiple models** share overlapping features (e.g., 5+ models using same user features)
- ✅ **Real-time inference** required (< 100ms latency SLA)
- ✅ **Frequent retraining** (weekly/monthly) → need reproducible feature pipelines
- ✅ **Large teams** (10+ ML engineers) → prevent duplicate work
- ❌ **Single model**, batch predictions only → simpler alternatives suffice

### Post-Silicon Use Cases
- **Wafer-Level Features**: Spatial statistics (die neighbors, edge vs center)
- **Temporal Features**: Test parameter trends across lots (process drift detection)
- **Multi-Site Features**: Correlation between wafer test and final test results
- **Equipment Features**: Tester calibration metadata joined with parametric data

### Tool Comparison

| **Tool** | **Best For** | **Strengths** | **Limitations** |
|---------|-------------|--------------|----------------|
| **Feast** | Open-source, Kubernetes-native | Free, active community, flexible | Requires DevOps expertise |
| **Tecton** | Enterprise, real-time streaming | Managed service, great UI | Expensive, vendor lock-in |
| **Hopsworks** | On-prem, regulated industries | Full control, feature monitoring | Complex setup |
| **AWS SageMaker** | AWS-native workloads | Seamless AWS integration | AWS lock-in, limited customization |
| **Databricks** | Spark-heavy pipelines | Unity Catalog integration | Spark overhead for simple features |

### Performance Benchmarks (Typical)
- **Offline Store**: 100K samples in 2-10 seconds (Parquet/BigQuery)
- **Online Store**: p99 latency < 10ms (Redis), < 50ms (DynamoDB)
- **Materialization**: 1M features in 5-30 minutes (Spark-based)

### Next Steps
- **Notebook 109**: ML Pipelines (orchestrate feature generation + model training)
- **Advanced**: Streaming features with Kafka/Flink, feature encryption, multi-region replication

---

**Remember**: *Feature stores are infrastructure. Build when complexity justifies the investment!* 🏗️